# Mapeamento de possíveis candidaturas laranjas

No decorrer deste trabalho, iremos trabalhar com os dataset's do TSE referentes aos resultados das eleições municipais de 2020.

O art. 10, § 3º, da Lei das Eleições (Lei 9.504 de 30 de Setembro de 1997), determina que, pelo menos, 30% (trinta por cento) das candidaturas efetivamente lançadas por um partido político seja destinada ao gênero oposto ao da maioria.

O objetivo do nosso trabalho é mapear possíveis candidaturas laranjas, tendo como critério a ausência de votos em candidatos deferidos para disputar o pleito.

### Importando as dependências e configurando a exibição dos DataFrames

In [1]:
import geopandas as gpd
import pandas as pd
import json
import matplotlib.pyplot as plt
from branca.colormap import linear
import folium
import os
br_estados = 'dados/br_states.json'
geo_json_data = json.load(open(br_estados))
pd.set_option('display.max_columns', None)

Abaixo iremos importar os dados dos candidatos e dos resultados nominais por município.
Os dados podem ser acessados no link: https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

Entretanto, não iremos extrair todos os dados dos dataset's. Abaixo definiremos uma lista de colunas que serão utilizadas, pois são úteis para os objetivos do estudo

In [2]:
colunasCandidatos = ['SQ_CANDIDATO','NR_IDADE_DATA_POSSE', 'DS_GENERO', 'DS_GRAU_INSTRUCAO','DS_COR_RACA','DS_OCUPACAO']
colunasResultados = ['NM_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_MUNICIPIO','SG_UF','SQ_CANDIDATO','DS_CARGO','DS_SITUACAO_CANDIDATURA','DS_DETALHE_SITUACAO_CAND','SG_PARTIDO','QT_VOTOS_NOMINAIS']

In [ ]:
dfCandidatos = pd.read_csv('dados/candidatos/consulta_cand_2020_BRASIL.csv', delimiter=';', encoding='latin-1', usecols = colunasCandidatos)

In [ ]:
dfResultados = pd.read_csv('dados/resultados/votacao_candidato_munzona_2020_BRASIL.csv', delimiter = ';', encoding = 'latin-1', usecols = colunasResultados)

Agora, podemos observar os dados selecionados dos candidatos. Filtramos apenas os dados pertinentes para o nosso estudo. O nome de possíveis candidatos "laranjas" foram removidos, por não são informações pertinentes.

In [ ]:
dfCandidatos.head()

### Filtrando os candidatos que não possuíram nenhum voto

Neste momento, iremos preparar os dados para identificar os possíveis candidatos de fachada.

O dataset não possui uma linha para cada candidato. Em diversos casos, várias linhas contabilizam a quantidade de votos parcial de um candidato. 

Para remover este problema, agruparemos os candidatos de acordo com o total de seus votos.

In [ ]:
grupoVotosTotais = dfResultados.groupby('SQ_CANDIDATO').sum()
grupoVotosTotais.head()

Agora possuímos o valor total de votos do candidato nesta coluna. 

Iremos adicionar esta coluna ao DataFrame principal para facilitar nosso trabalho.

In [ ]:
dfResultados = pd.merge(dfResultados, grupoVotosTotais, on = 'SQ_CANDIDATO')

In [ ]:
dfResultados.head(3)

Acima, possuímos o DataFrame indicando a quantidade totais de votos do candidato. 

Agora, iremos remover a coluna de votos parciais.

In [ ]:
dfResultados = dfResultados.drop(columns=['QT_VOTOS_NOMINAIS_x'])
dfResultados.columns = ['SG_UF', 'NM_MUNICIPIO', 'DS_CARGO', 'SQ_CANDIDATO', 'NM_CANDIDATO','NM_URNA_CANDIDATO', 'DS_SITUACAO_CANDIDATURA', 'DS_DETALHE_SITUACAO_CAND', 'SG_PARTIDO', 'QT_VOTOS_NOMINAIS']

Abaixo iremos remover os duplicados, que existem em decorrência das múltiplas linhas que continham os dados parcias de votação.

In [ ]:
dfResultados.drop_duplicates(inplace = True)

#### Critério de avaliação de possíveis candidatos laranjas

Iremos mapear neste estudo, todos os candidatos ao legislativo que não possuíram nenhum voto, mesmo estando apto a ter seu nome na urna, e com o processo de candidatura deferido pela Justiça Eleitoral.

Para isso, iremos remover todos os candidatos que não se encontram neste posição.

In [ ]:
dfCandidatosZeroVoto = dfResultados.query("QT_VOTOS_NOMINAIS == 0 and DS_SITUACAO_CANDIDATURA == 'APTO' and DS_DETALHE_SITUACAO_CAND == 'DEFERIDO'")
dfCandidatosZeroVoto.head(3)

Com isso, podemos começar a extrair os primeiros resultados do estudo. Iremos armazenar alguns deles para exibi-los na seção de resultados.

Extraindo o relatório da quantidade de partidos com candidatos sem votos:

In [ ]:
resultadosPartidos = dfCandidatosZeroVoto.groupby('SG_PARTIDO')['SG_PARTIDO'].count()
resultadosPartidos.sort_values(inplace = True, ascending = False)

### Extraindo resultados referentes aos candidatos.

Para extrair dados sobre os candidatos, precisaremos relacionar a base de dados dos candidatos sem votos com a base de dados de detalhes sobre o concorrente.

O método "merge" abaixo foi utilizado para unir e relacionar estes dados.

In [ ]:
dfCandidatosZeroVoto = pd.merge(dfCandidatosZeroVoto, dfCandidatos, on = "SQ_CANDIDATO")

Abaixo, os valores serão armazenados para exibição futura.

In [ ]:
grauEscolaridade = dfCandidatosZeroVoto.groupby(['DS_GRAU_INSTRUCAO'])['DS_GRAU_INSTRUCAO'].count().sort_values()
estado = dfCandidatosZeroVoto.groupby(['SG_UF'])['SG_PARTIDO'].count().sort_values()
genero = dfCandidatosZeroVoto.groupby(['DS_GENERO'])['DS_GENERO'].count().sort_values()
etnia = dfCandidatosZeroVoto.groupby(['DS_COR_RACA'])['DS_COR_RACA'].count().sort_values()
ocupacao = dfCandidatosZeroVoto.groupby(['DS_OCUPACAO'])['DS_OCUPACAO'].count().sort_values()

### Extraindo resultados referentes às regiões e estados.

Este trabalho tem como objetivo mostrar em quais regiões e estados do Brasil estas candidaturas se concentram.

Para isso, utilizaremos dados do IBGE que relacionam a posição da região (latitude e longitude) com o nome do município. 

Estes dados são fornecidos em formatos não suportados pela biblioteca Pandas. Por isso, utilizaremos o GeoPandas para leitura destas informações.

In [ ]:
dfLocal = gpd.read_file('dados/ShapefileIBGE/BR_Localidades_2010_v1.shp')
dfLocal.query("NM_CATEGOR == 'CIDADE'", inplace = True)
dfLocal = dfLocal[['NM_DISTRIT', 'NM_UF', 'NM_CATEGOR', 'LONG', 'LAT']]
dfLocal.columns = ['NM_MUNICIPIO', 'NM_UF', 'NM_CATEGOR', 'LONG', 'LAT']

Após extrair o DataFrame com as localidades, iremos relacionar estes dados ao DataFrame de candidatos sem votos.

In [ ]:
zeroVotosLocal = pd.merge(dfCandidatosZeroVoto, dfLocal, on = 'NM_MUNICIPIO')
zeroVotosLocal = zeroVotosLocal[['LAT', 'LONG']]
zeroVotosUF = dfCandidatosZeroVoto['SG_UF']

## Exibição dos resultados

In [ ]:
def plotGrafico(fator, size, groupby, color, fontsize):
    plt.rcParams.update({'font.size': fontsize})
    plt.figure(figsize = size)
    plt.xlabel(f"\nQuantidade de candidatos sem votos")
    plt.ylabel(f"{fator}")
    plt.title(f"\nQuantidade de candidatos por {fator}\n")
    plt.barh(groupby.index, groupby.values, color = color)
    plt.show()

In [ ]:
plotGrafico('Partido', (22,15), resultadosPartidos, 'red', 22)

In [ ]:
plotGrafico('Grau de escolaridade', (20,10), grauEscolaridade, 'blue', 16)

In [ ]:
plotGrafico('UF', (20,15), estado, 'green', 18)

In [ ]:
plotGrafico('Sexo', (8,4), genero, 'black', 13)

In [ ]:
plotGrafico('Etnia', (8,4), etnia, 'brown', 13)

Por fim, temos a represenção por estado da quantidade de candidatos sem votos bruta abaixo.

In [ ]:
zeroVotosUF = zeroVotosUF.to_frame()
zeroVotosUF = zeroVotosUF.groupby('SG_UF').SG_UF.count().to_frame()
colormap = linear.OrRd_09.scale(0,614)
zeroVotosUF = zeroVotosUF.to_dict()
zeroVotosUF = zeroVotosUF['SG_UF']
zeroVotosUF['DF'] = 0
mapa = folium.Map(width=900, height = 600, location=[-13.77972, -47.92972], zoom_start = 4)
folium.GeoJson(geo_json_data, name='2020', style_function=lambda feature: 
        {'fillColor': colormap(zeroVotosUF[feature['id']]),
        'color': 'black',
        'weight': 0.5,}
).add_to(mapa)
colormap.caption = 'Quantidade de candidatos sem nenhum voto por estado'
colormap.add_to(mapa)
folium.LayerControl(collapsed = True).add_to(mapa)
mapa